In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import datetime
import sys
import re
import gc
import glob

import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from utils import logger_func
logger = logger_func()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
from sklearn.metrics import mean_squared_error


path_list = glob.glob('../stack/*.gz')
import pickle
import datetime
from tqdm import tqdm
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term']

2019-02-16 17:20:58,397 utils 400 [INFO]    [logger_func] start 


In [35]:
#========================================================================
# CV CHECKER 
# Data Load
def get_cv_score(model_path):
    base = utils.read_df_pkl('../input/base_term*')[[key, target]].set_index(key)
    train = base[~base[target].isnull()]

    tmp = utils.read_pkl_gzip(model_path)
    if 'pred_mean' in tmp.columns:
        pred_col = 'pred_mean'
    elif 'prediction' in tmp.columns:
        pred_col = 'prediction'
    tmp = tmp.reset_index()[[key, pred_col]].set_index(key)
    
    train['pred'] = tmp[pred_col]

    y_train = train[target].values
    y_pred = train['pred'].values
    score = np.sqrt(mean_squared_error(y_train, y_pred))
    print(f"CV: {score}")
    return score
#========================================================================


#========================================================================
# 相関

def get_corr_ensemble(path, base, out_part = ''):
    '''
    1. ベースモデルとの相関を個別に取得する
    2. ベースモデルにpath_listのモデルの予測をjoinし、相関行列を取得する
    '''
    
    if key in base.columns:
        base = base.set_index(key)
    
    #========================================================================
    # 個別に相関を見ていく場合
    if str(type(path)).count('str'):
    
        if path.count('CV1'):
            return 0, ''
        if path.count('binary'):
            return 0, ''
        tmp = utils.read_pkl_gzip(path)
    
        try:
            tmp = tmp.reset_index()[[key, 'pred_mean']].set_index(key)
            base['tmp_pred'] = tmp.reset_index()['pred_mean']
        except KeyError:
            tmp = tmp.reset_index()[[key, 'prediction']].set_index(key)
            base['tmp_pred'] = tmp['prediction']
    
        if path.count('LB'):
            try:
                cv2 = re.search(r'CV([^/.]*)_LB.gz', path.replace('.', '-')).group(1)
            except AttributeError:
                return 0, ''
        else:
            cv2 = re.search(r'CV([^/.]*)', path.replace('.', '-')).group(1)
    
        corr = np.corrcoef(base['base_pred'], base['tmp_pred'].values).min()
    
        if corr>0 and corr<0.98:
            logger.info(f"CORR: {corr} | CV{cv1[:6]} vs CV{cv2[:6]}")
            
            return corr, path
        else:
            return 0, ''
    #========================================================================
    
    #========================================================================
    # まとめて相関行列を見たい場合
    elif str(type(path)).count('list'):
        path_list = path
        for path in path_list:
        
            if out_part=='all':
                if path.count('CV-1') or path.count('binary'):
                    continue
            elif out_part=='rm_out':
                if path.count('CV-3') or path.count('binary'):
                    continue
            elif out_part=='binary':
                if path.count('CV-3') or path.count('CV-1'):
                    continue
            tmp = utils.read_pkl_gzip(path)

            if 'pred_mean' in tmp.columns:
                pred_col = 'pred_mean'
            elif 'prediction' in tmp.columns:
                pred_col = 'prediction'
                
            tmp = tmp.reset_index()[[key, pred_col]].set_index(key)
                
            if path.count('LB'):
                try:
                    cv2 = re.search(r'CV([^/.]*)_LB', path.replace('.', '-')).group(1)
                except AttributeError:
                    print(path.replace('.', '-'))
                    continue
            else:
                cv2 = re.search(r'CV([^/.]*)', path.replace('.', '-')).group(1)
                
            base[f'pr_{cv2[:9]}'] = tmp[pred_col]
            
        if out_part=='rm_out':
            base = base[base[target]>-30]
        drop_cols = [col for col in base.columns if col.count('term') or col.count('no_out_flg') or col in ignore_list or col.count('clf')]
        base.drop(drop_cols, axis=1, inplace=True)
        base.sort_index(axis=1, inplace=True)
        mx_corr = base.corr(method='pearson')
#         mx_cor = mx_corr.drop(drop_cols, axis=1).loc[~mx_corr.index.isin(drop_cols)]
        
        return mx_corr
#========================================================================

In [47]:
is_clf = 0
base = utils.read_df_pkl('../input/base_no_out*')[[key, target, 'clf_pred', 'no_out_flg']].set_index(key)
# LB3664 = utils.read_pkl_gzip('../ensemble/good_submit_ensemble/0215_083_stack_submit_OUT29-7593_CV3-61266_LB3-664.gz').set_index(key)
# base['lb3664'] = LB3664['prediction']
#========================================================================
# rm_out以外の部分の相関を見る
# base = base[base['clf_pred']>=0.01]
#========================================================================
# ens_list = glob.glob('../stack/*lgb*all*CV3*.gz')
# ens_list = glob.glob('../model/LB3664_set/*_lgb_*.gz')
ens_list = glob.glob('../ensemble/dir_stack_blend/tmp/*_lgb_*.gz')
out_part = 'all'
# ens_list += glob.glob('../stack/0213_2*no_out*.gz') + glob.glob('../stack/0214_*no_out*.gz')

if is_clf:
    out_part = 'binary'
#     ens_list = glob.glob('../stack/0214*NN*-clf*.gz')
    ens_list = glob.glob('../ensemble/binary_ensemble/*.gz')
    base[target] = base[target].map(lambda x: 1 if x<-30 else 0)
top_corr = get_corr_ensemble(path=ens_list, base=base, out_part=out_part)
print(top_corr.shape)
display(top_corr.head())
# top_corr.to_csv('../output/0214_07_elo_rm_out_corr_ensemble_heatmap.csv', index=True)

100%|██████████| 1/1 [00:00<00:00,  4.29it/s]


(23, 23)


,pr_3-2381280,pr_3-2400031,pr_3-6206463,pr_3-6216744,pr_3-6221003,pr_3-6221890,pr_3-6226545,pr_3-6236254,pr_3-6241143,pr_3-6246353,pr_3-6248860,pr_3-6251774,pr_3-6256427,pr_3-627555,pr_3-6297296,pr_3-6333204,pr_3-6335126,pr_3-6402002,pr_3-6430089,pr_3-6431931,pr_3-6435566,pr_3-6514559,pr_3-6536747
pr_3-2381280,1.000000,0.984552,0.958261,0.954465,0.950571,0.948359,0.948427,0.949438,0.950181,0.948058,0.956810,0.935781,0.931313,0.966633,0.949348,0.942884,0.935625,0.935378,0.931024,0.927631,0.933077,0.924552,0.909176
pr_3-2400031,0.984552,1.000000,0.954855,0.952100,0.946157,0.944078,0.944157,0.947065,0.946844,0.945385,0.954648,0.931766,0.926826,0.966749,0.945790,0.940391,0.935481,0.933756,0.927905,0.932139,0.935958,0.929366,0.912180
pr_3-6206463,0.958261,0.954855,1.000000,0.979583,0.983439,0.981261,0.981506,0.974725,0.982675,0.971927,0.984951,0.964666,0.957821,0.978861,0.968850,0.964006,0.957569,0.952506,0.945155,0.941713,0.942556,0.934902,0.921691
pr_3-6216744,0.954465,0.952100,0.979583,1.000000,0.973080,0.971300,0.971371,0.967099,0.972065,0.983273,0.975268,0.955712,0.950126,0.982624,0.965467,0.976818,0.951645,0.950574,0.957258,0.946569,0.948470,0.934354,0.922376
pr_3-6221003,0.950571,0.946157,0.983439,0.973080,1.000000,0.982309,0.981505,0.966404,0.978338,0.965814,0.981949,0.959637,0.958703,0.969759,0.961809,0.957759,0.946885,0.943052,0.937326,0.930928,0.936271,0.925489,0.911784


In [49]:
top_corr[top_corr<1].max()

pr_3-2381280    0.984552
pr_3-2400031    0.984552
pr_3-6206463    0.984951
pr_3-6216744    0.983273
pr_3-6221003    0.983439
pr_3-6221890    0.982309
pr_3-6226545    0.982136
pr_3-6236254    0.976131
pr_3-6241143    0.982675
pr_3-6246353    0.983273
pr_3-6248860    0.984951
pr_3-6251774    0.984067
pr_3-6256427    0.984067
pr_3-627555     0.982624
pr_3-6297296    0.978292
pr_3-6333204    0.976818
pr_3-6335126    0.974098
pr_3-6402002    0.973060
pr_3-6430089    0.969421
pr_3-6431931    0.973092
pr_3-6435566    0.970644
pr_3-6514559    0.962821
pr_3-6536747    0.954844
dtype: float64

In [44]:
#========================================================================
# Get Low Corr Model For Ensemble
select_list = []
thres_corr = 0.985
base_model = top_corr.iloc[0].index[0]
select_list.append(base_model)

def ensemble_corr_checker(model):
    df_corr = top_corr.loc[model].reset_index()
    
    current_idx = list(top_corr.index).index(model)
    df_corr = df_corr.iloc[current_idx:, ]
    best_cv = 100
    best_corr = 0
    for _, rows in df_corr.iterrows():
        cv = rows[0]
        corr = rows[1]
        
        if cv not in select_list and corr<thres_corr:
            tmp = top_corr.loc[select_list, cv]
            cnt = tmp[tmp>=thres_corr].sum()
            if cnt==0:
                return cv, corr
    
    return '', 0

# cv, corr = ensemble_corr_checker(base_model)
# if corr>=thres_corr:
#     continue
    
is_no_stack = 0
is_loop = True
while is_loop:
    for model in select_list:    
        cv, corr = ensemble_corr_checker(base_model)
        if len(select_list)==1:
            if corr:
                select_list.append(cv)
        elif len(select_list)>1:
            cv, corr = ensemble_corr_checker(select_list[-1])
            if corr:
                select_list.append(cv)
            else:
                is_loop=False
#========================================================================

In [43]:
print(select_list)
df_select = top_corr.loc[select_list, select_list]
# tmp = df_select['lb3664'].iloc[:9]
select_list = list(tmp.index)
select_list

['pr_3-2381280', 'pr_3-2400031', 'pr_3-6206463', 'pr_3-6216744', 'pr_3-6221003', 'pr_3-6221890', 'pr_3-6226545', 'pr_3-6236254', 'pr_3-6241143', 'pr_3-6246353', 'pr_3-6248860', 'pr_3-6251774', 'pr_3-6256427', 'pr_3-627555', 'pr_3-6297296', 'pr_3-6333204', 'pr_3-6335126', 'pr_3-6402002', 'pr_3-6430089', 'pr_3-6431931', 'pr_3-6435566', 'pr_3-6514559', 'pr_3-6536747']


['lb3664',
 'pr_3-6206463',
 'pr_3-6221003',
 'pr_3-6221890',
 'pr_3-6226545',
 'pr_3-6241143',
 'pr_3-6248860',
 'pr_3-6251774',
 'pr_3-6256427']

In [46]:
# ================================================================
# 相関行列から移動するpathを決める
import shutil
for path in ens_list:
    for sel in select_list:
#         sel_key = sel[3:].replace('-', '.')
        sel_key = sel[3:]
        if path.count(sel_key):
#             shutil.move(path, '../ensemble/lgb_ensemble/')
            try:
#                 shutil.move(path, '../ensemble/dir_stack_blend/')
                shutil.move(path, '../ensemble/dir_stack_blend/tmp/')
            except FileNotFoundError:
                pass
            pass
        else:
            pass
#             shutil.move(path, '../ensemble/used_stack/')
# ================================================================

100%|██████████| 1/1 [00:00<00:00,  2.90it/s]


,target,clf_pred,no_out_flg
card_id,,,
C_ID_92a2005557,-0.820283,0.000444,1.0
C_ID_3d0044924f,0.392913,0.007831,0.0
C_ID_d639edf6cd,0.688056,0.004074,0.0
C_ID_186d6a6901,0.142495,0.000797,0.0
C_ID_cdbd2c0db2,-0.159749,0.000251,1.0
